# Scrape Master Schedule File

In [50]:
import os
import requests
from bs4 import BeautifulSoup
import csv

# URL for the ESPN NFL schedule grid
base_url = "https://www.espn.com/nfl/schedulegrid/_/year/"

# Years for which you want to scrape data
years = [2021, 2022, 2023]
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Directory to save the combined CSV file
save_directory = "datasets"
combined_csv_filename = os.path.join(save_directory, "master_schedule.csv")

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Flag to indicate whether to include headers or not
include_headers = True

# Open the file in 'w' mode to overwrite if it already exists
with open(combined_csv_filename, 'w', newline='', encoding='utf-8') as combined_csv_file:
    combined_csv_writer = csv.writer(combined_csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for year in years:
        url = f"{base_url}{year}"

        # Send a GET request to the URL
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the tables on the page
            tables = soup.find_all('table')

            # Process each table
            for index, table in enumerate(tables):
                # Extract data from the table as needed
                rows = table.find_all('tr')

                # Prepare data for writing to the combined CSV file
                data_rows = []
                for row in rows[1:]:
                    data_rows.append([year] + [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])])

                # Include headers only for the first iteration
                if include_headers:
                    combined_csv_writer.writerow([cell.get_text(strip=True) for cell in rows[0].find_all('th')])
                    include_headers = False  # Set to False after writing headers

                combined_csv_writer.writerows(data_rows[1:])  # Exclude the first row

            print(f"Year {year} data appended to {combined_csv_filename}\n{'='*40}\n")

        else:
            print(f"Failed to retrieve data for the year {year}. Status code: {response.status_code}")

print(f"All data combined and saved to {combined_csv_filename}")

Year 2021 data appended to datasets/master_schedule.csv

Year 2022 data appended to datasets/master_schedule.csv

Year 2023 data appended to datasets/master_schedule.csv

All data combined and saved to datasets/master_schedule.csv


# Clean data

In [51]:
import pandas as pd

df = pd.read_csv("datasets/master_schedule.csv", header=None)

# List of new column names
new_column_names = ['YEAR', 'TEAM', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']

# Use the rename method
df.rename(columns=dict(zip(df.columns, new_column_names)), inplace=True)

df = df.replace('@', '', regex=True)

df.to_csv("datasets/master_schedule.csv", index=False)

df.head(10)

,YEAR,TEAM,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,2021,ARI,TEN,MIN,JAX,LAR,SF,CLE,HOU,GB,SF,CAR,SEA,BYE,CHI,LAR,DET,IND,DAL,SEA
1,2021,ATL,PHI,TB,NYG,WSH,NYJ,BYE,MIA,CAR,NO,DAL,NE,JAX,TB,CAR,SF,DET,BUF,NO
2,2021,BAL,LV,KC,DET,DEN,IND,LAC,CIN,BYE,MIN,MIA,CHI,CLE,PIT,CLE,GB,CIN,LAR,PIT
3,2021,BUF,PIT,MIA,WSH,HOU,KC,TEN,BYE,MIA,JAX,NYJ,IND,NO,NE,TB,CAR,NE,ATL,NYJ
4,2021,CAR,NYJ,NO,HOU,DAL,PHI,MIN,NYG,ATL,NE,ARI,WSH,MIA,BYE,ATL,BUF,TB,NO,TB
5,2021,CHI,LAR,CIN,CLE,DET,LV,GB,TB,SF,PIT,BYE,BAL,DET,ARI,GB,MIN,SEA,NYG,MIN
6,2021,CIN,MIN,CHI,PIT,JAX,GB,DET,BAL,NYJ,CLE,BYE,LV,PIT,LAC,SF,DEN,BAL,KC,CLE
7,2021,CLE,KC,HOU,CHI,MIN,LAC,ARI,DEN,PIT,CIN,NE,DET,BAL,BYE,BAL,LV,GB,PIT,CIN
8,2021,DAL,TB,LAC,PHI,CAR,NYG,NE,BYE,MIN,DEN,ATL,KC,LV,NO,WSH,NYG,WSH,ARI,PHI
9,2021,DEN,NYG,JAX,NYJ,BAL,PIT,LV,CLE,WSH,DAL,PHI,BYE,LAC,KC,DET,CIN,LV,LAC,KC


# Join

In [52]:
import pandas as pd

# Assuming dataset_a and dataset_b are your two DataFrames
# Replace these with the actual names of your DataFrames
dataset_a = pd.DataFrame({
    'WEEK': [1, 2, 3],
    'TEAM': ['ARI', 'ATL', 'BAL'],
    'SCORE': [10, 15, 20],
    'YEAR': [2021, 2022, 2021]
})

dataset_b = pd.DataFrame({
    'TEAM': ['ARI', 'ATL', 'BAL'],
    '1': ['TEN', 'PHI', 'LV'],
    '2': ['MIN', 'TB', 'KC'],
    '3': ['JAC', 'NYG', 'DET'],
    'YEAR': [2021, 2022, 2021]
})

weekly_scoring = pd.read_csv("datasets/weekly_scoring.csv")
master_schedule = pd.read_csv("datasets/master_schedule.csv")

dataset_a = weekly_scoring
dataset_b = master_schedule

# Reshape dataset_b to have columns ['week', 'team', 'opponent']
melted_dataset_b = pd.melt(dataset_b, id_vars=['YEAR', 'TEAM'], var_name='WEEK', value_name='OPP')

# Convert 'WEEK' column to integer
melted_dataset_b['WEEK'] = melted_dataset_b['WEEK'].astype(int)

# Merge datasets based on 'week', 'team', and 'year'
merged_dataset = pd.merge(dataset_a, melted_dataset_b, on=['WEEK', 'TEAM', 'YEAR'], how='left')


# Display the merged DataFrame
merged_dataset.to_csv("datasets/weekly_scoring.csv")

,POS RANK,PLAYER,PASSING CMP,PASSING ATT,PASSING PCT,PASSING YDS,PASSING Y/A,PASSING TD,PASSING INT,PASSING SACKS,...,RECEIVING LG,RECEIVING 20+,RECEIVING TD,RUSHING Y/A,RUSHING LG,RUSHING 20+,DATE,WEIGHT,TEAM,OPP
0,1,Kyler Murray (ARI),21.0,32.0,65.6,289.0,9.0,4.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04,4.132631e-46,ARI,TEN
1,2,Patrick Mahomes II (KC),27.0,36.0,75.0,337.0,9.4,3.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04,4.132631e-46,KC,CLE
2,3,Jared Goff (DET),38.0,57.0,66.7,338.0,5.9,3.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04,4.132631e-46,DET,SF
3,4,Jameis Winston (NO),14.0,20.0,70.0,148.0,7.4,5.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04,4.132631e-46,NO,GB
4,5,Tom Brady (FA),32.0,50.0,64.0,379.0,7.6,4.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04,4.132631e-46,FA,NaN
5,6,Jalen Hurts (PHI),27.0,35.0,77.1,264.0,7.5,3.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04,4.132631e-46,PHI,ATL
6,7,Dak Prescott (DAL),42.0,58.0,72.4,403.0,6.9,3.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04,4.132631e-46,DAL,TB
7,8,Russell Wilson (DEN),18.0,23.0,78.3,254.0,11.0,4.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04,4.132631e-46,DEN,NYG
8,9,Derek Carr (NO),34.0,56.0,60.7,435.0,7.8,2.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04,4.132631e-46,NO,GB
9,10,Matthew Stafford (LAR),20.0,26.0,76.9,321.0,12.3,3.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04,4.132631e-46,LAR,CHI
